Установка библиотеки Transformers версии 4.37.0 или выше для работы с предобученными моделями из Hugging Face.

In [ ]:
!pip install -q transformers>=4.37.0

Импорт библиотек: `torch` для работы с тензорами и вычислениями на GPU, а также `AutoTokenizer` и `AutoModelForCausalLM` из `transformers` для загрузки токенизатора и модели автогенерации текста.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

Импорт модулей для загрузки файлов в Google Colab: `files` — для взаимодействия с локальной файловой системой пользователя, `io` — для работы с потоками ввода-вывода.

In [ ]:
from google.colab import files
import io

Загрузка предобученной модели и токенизатора `Qwen2-1.5B-Instruct` из Hugging Face. Модель переносится на выбранное ранее устройство (`GPU` или `CPU`).

In [ ]:
# Проверка и выбор устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


Загрузка модели и токенизатора `Qwen2-1.5B-Instruct` из Hugging Face с использованием кастомного кода (опция `trust_remote_code=True`). Модель загружается на выбранное устройство (`device`).

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "Qwen/Qwen2-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Создание словаря `video_data`, содержащего информацию о видео: идентификатор, имя файла и подробное текстовое описание. Затем извлекается описание из словаря для дальнейшего использования.

In [ ]:
import json

video_data = {
    "id": 1,
    "filename":"00ba8a28-b6af-40b4-9124-ec10c9863b28.mp4",
    "description":"Тема видео: Обзор и демонстрация характеристик беспроводных наушников Picun F6.\n\nСценарий:\n\n1. Вступление:\n   - Мужчина, одетый в легкую, синюю футболку, представляет продукт и говорит о удивительных качествах китайских технологий.\n   - На экране появляется текст «Китайцы не перестают удивлять» с эмодзи, подчеркивая интригу и интерес к продукту.\n\n2. Появление и открытие коробки:\n   - Мужчина показывает коробку с изображением наушников, которая открывается, откровенно раскрывая их дизайн.\n   - На коробке написано «Picun F6», что является моделью наушников.\n\n3. Демонстрация наушников:\n   - Мужчина демонстрирует различные части наушников, акцентируя внимание на их строении и функциональности.\n   - Он показывает, как регулировать длину шейлова, подчеркивая удобство и гибкость.\n\n4. Изображение наушников на человеке:\n   - Наушники демонстрируются на человеке, а на экране отображается цена наушников — 51$.\n   - Мужчина демонстрирует их удобство в ношении на улице.\n\n5. Показ навигационной панели:\n   - Мужчина демонстрирует навигационную панель на наушниках, позволяющую настроить различные параметры, такие как эквалайзер и шумоподавление.\n   - Панель показывает настроенные параметры и доступные режимы.\n\n6. Применение приложения для управления:\n   - Мужчина демонстрирует использование мобильного приложения для управления наушниками.\n   - Он показывает, как добавить наушники к устройству, настроить звук и другие параметры.\n\n7. Заключение:\n   - В заключение, мужчина подводит итоги, подчеркивая преимущества"
  }

description = video_data["description"]

Импорт модуля `re` для работы с регулярными выражениями, который может использоваться для обработки и анализа текста.

In [ ]:
import re

Определение путей к входному и выходному JSON-файлам: `INPUT_JSON_PATH` — для загрузки исходных данных, `OUTPUT_JSON_PATH` — для сохранения результатов с хэштегами.

In [ ]:
INPUT_JSON_PATH = "/content/merged_ordered_3600_5500.json"  # Путь к исходному JSON
OUTPUT_JSON_PATH = "/content/hashtags_3600_5500.json"  # Путь к выходному JSON

In [ ]:
DEVICE = "cuda"  # или "cpu"

Определение функции `generate_hashtags`, которая генерирует список уникальных осмысленных хэштегов на русском языке на основе текстового описания видео.
Используется LLM-модель для генерации, затем с помощью регулярных выражений извлекаются хэштеги, удаляются дубли, и сохраняется порядок появления.

In [ ]:
def generate_hashtags(description: str) -> list[str]:
    prompt = f"""
Video description:
\"{description}\"

Task:
Generate a list of **only informative hashtags in Russian** that reflect the content.
"""
    # Assuming tokenizer, model, and DEVICE are defined elsewhere
    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    outputs = model.generate(**inputs, max_new_tokens=50)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    hashtags = re.findall(r"#\w+", generated_text)
    tags = re.findall(r"#(\w+)", generated_text)  # Fixed: Use generated_text instead of text
    # Remove duplicates while preserving order
    seen = set()
    uniq = []
    for t in tags:
        if t not in seen:
            seen.add(t)
            uniq.append(t)
    return uniq

description = video.get("description", '') or ''  # Fallback to '' if None
transcription = video.get("transcription", '') or ''  # Fallback to '' if None
combined_text = description + transcription

  # Print combined text for debugging

# Generate hashtags for the video
tags = generate_hashtags(combined_text)

video["tags"] = tags

# Output the result to the console


print(" ".join(video["tags"]))

#Запуск тегирования

Загрузка входного JSON-файла с данными о видео. Для каждого видео генерируются хэштеги на основе описания и транскрипции, после чего результат сохраняется в выходной JSON-файл.

In [ ]:
# Загрузка данных
with open(INPUT_JSON_PATH, "r", encoding="utf-8") as f:
    videos = json.load(f)

# Обработка и добавление хештегов
for video in videos:
    desc = video.get("description", "") or ''
    trans = video.get('transcription', '') or ''
    video["tags"] = generate_hashtags(desc + trans)

# Сохранение результата
with open(OUTPUT_JSON_PATH, "w", encoding="utf-8") as f:
    json.dump(videos, f, ensure_ascii=False, indent=2)

print(f"Хештеги добавлены. Файл сохранён как: {OUTPUT_JSON_PATH}")

Функция `generate_hashtags` генерирует уникальные хэштеги на русском языке, извлекая их из текста, созданного моделью на основе описания и транскрипции видео.
Проверяется наличие полей `description` и `transcription`, объединяется текст, затем генерируются и сохраняются хэштеги в поле `tags`.
Результат выводится в консоль.

In [ ]:
def generate_hashtags(description: str) -> list[str]:
    prompt = f"""
        Video description:
        \"{description}\"

        Task:
        Generate a list of **only informative hashtags in Russian** that reflect the content.
    """
    # Assuming tokenizer, model, and DEVICE are defined elsewhere
    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    outputs = model.generate(**inputs, max_new_tokens=50)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    hashtags = re.findall(r"#\w+", generated_text)
    tags = re.findall(r"#(\w+)", generated_text)  # Fixed: Use generated_text instead of text
    # Remove duplicates while preserving order
    seen = set()
    uniq = []
    for t in tags:
        if t not in seen:
            seen.add(t)
            uniq.append(t)
    return uniq

description = video.get("description", '') or ''  # Fallback to '' if None
transcription = video.get("transcription", '') or ''  # Fallback to '' if None
combined_text = description + transcription

  # Print combined text for debugging

# Generate hashtags for the video
tags = generate_hashtags(combined_text)

video["tags"] = tags

# Output the result to the console


print(" ".join(video["tags"]))


Щенки Пометы Цены Шоу Брит Пэт К
